So we know from Notebook 10 that trying to make it learn one set of logits per batch is good and gets features right.

How can we make the logits consistent over batches?

Will collect the gradients and gumbel selected values after 5 epochs in each mode. The two modes will be the behavior before burn-in and the behavior after mode-in.

Will follow Notebook 10 with slight modifications (described right before). Remember in Notebook 10, we explored behaviors when all the features were real vs when half the features were noise.

In [1]:
import torch


from torch import nn
from torch.autograd import Variable
from torch.nn import functional as F

import numpy as np

import matplotlib.pyplot as plt
#from sklearn.manifold import TSNE

#import math

#import gc

from utils import *

from sklearn.preprocessing import MinMaxScaler

from scipy.stats import pearsonr

import seaborn as sns

In [2]:
torch.manual_seed(0)
np.random.seed(0)

In [3]:
# really good results for vanilla VAE on synthetic data with EPOCHS set to 50, 
# but when running locally set to 10 for reasonable run times
n_epochs = 600
batch_size = 64
lr = 0.0001
b1 = 0.9
b2 = 0.999

In [4]:
cuda = True if torch.cuda.is_available() else False

Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor

device = torch.device("cuda:0" if cuda else "cpu")
#device = 'cpu'
print("Device")
print(device)

Device
cuda:0


In [5]:
D = 30
N = 10000
z_size = 8

In [6]:
torch.manual_seed(0)
np.random.seed(0)

latent_data = np.random.normal(loc=0.0, scale=1.0, size=N*z_size).reshape(N, z_size)

data_mapper = nn.Sequential(
    nn.Linear(z_size, 2 * z_size, bias=False),
    nn.Tanh(),
    nn.Linear(2 * z_size, D, bias = True),
    nn.ReLU()
).to(device)

data_mapper.requires_grad_(False)

latent_data = Tensor(latent_data)
latent_data.requires_grad_(False)

actual_data = data_mapper(latent_data)


for i in range(5):
    print(torch.sum(actual_data[i,:] != 0))

tensor(14, device='cuda:0')
tensor(18, device='cuda:0')
tensor(14, device='cuda:0')
tensor(17, device='cuda:0')
tensor(16, device='cuda:0')


add noiise

In [7]:
noise_features = torch.empty(N * D).normal_(mean=0,std=0.01).reshape(N, D).to(device)
noise_features.requires_grad_(False)

tensor([[ 0.0013,  0.0135,  0.0054,  ..., -0.0047,  0.0033, -0.0097],
        [ 0.0080, -0.0057,  0.0010,  ...,  0.0009, -0.0134,  0.0105],
        [-0.0103, -0.0029,  0.0185,  ..., -0.0133, -0.0037,  0.0134],
        ...,
        [ 0.0073, -0.0149, -0.0108,  ..., -0.0047, -0.0137,  0.0070],
        [ 0.0006, -0.0141, -0.0124,  ..., -0.0085,  0.0069, -0.0110],
        [-0.0159,  0.0177, -0.0087,  ..., -0.0076, -0.0009,  0.0078]],
       device='cuda:0')

In [8]:
actual_data = torch.cat([actual_data, noise_features], dim = 1)

In [9]:
actual_data.shape

torch.Size([10000, 60])

In [10]:
actual_data = actual_data.cpu().numpy()
scaler = MinMaxScaler()
actual_data = scaler.fit_transform(actual_data)

actual_data = Tensor(actual_data)

slices = np.random.permutation(np.arange(actual_data.shape[0]))
upto = int(.8 * len(actual_data))

train_data = actual_data[slices[:upto]]
test_data = actual_data[slices[upto:]]

In [11]:
global_t = 4

Example of what worked before

In [12]:
vae_gumbel_truncated = VAE_Gumbel(2*D, 100, 20, k = 3*z_size, t = global_t)
vae_gumbel_truncated.to(device)
vae_gumbel_trunc_optimizer = torch.optim.Adam(vae_gumbel_truncated.parameters(), 
                                                lr=lr, 
                                                betas = (b1,b2))

In [13]:
gradients_before_burnin = torch.zeros(train_data.shape[1]).to(device)
gradient_post_burn_in = torch.zeros(train_data.shape[1]).to(device)
subset_indices_before_burnin = torch.zeros(train_data.shape[1]).to(device)
subset_indices_post_burnin = torch.zeros(train_data.shape[1]).to(device)

for epoch in range(1, n_epochs+1):
    grads=train_truncated_with_gradients(train_data, vae_gumbel_truncated, 
                                                      vae_gumbel_trunc_optimizer, 
                                                      epoch, 
                                                      batch_size, 
                                                      Dim = 2*D)
    if epoch <=(n_epochs//5*4):
        gradients_before_burnin += grads
        with torch.no_grad():
            subset_indices_before_burnin += vae_gumbel_truncated.subset_indices.sum(dim = 0)
    if epoch > (n_epochs//5*4):
        gradient_post_burn_in += grads
        with torch.no_grad():
            subset_indices_post_burnin += vae_gumbel_truncated.subset_indices.sum(dim = 0)

Train Epoch: 1 [0/8000 (0%)]	Loss: 42.025238
Train Epoch: 1 [1280/8000 (16%)]	Loss: 41.364944
Train Epoch: 1 [2560/8000 (32%)]	Loss: 40.814362
Train Epoch: 1 [3840/8000 (48%)]	Loss: 40.379112
Train Epoch: 1 [5120/8000 (64%)]	Loss: 39.789692
Train Epoch: 1 [6400/8000 (80%)]	Loss: 39.276009
Train Epoch: 1 [7680/8000 (96%)]	Loss: 38.740120
====> Epoch: 1 Average loss: 40.2651
Train Epoch: 2 [0/8000 (0%)]	Loss: 38.672668
Train Epoch: 2 [1280/8000 (16%)]	Loss: 37.857777
Train Epoch: 2 [2560/8000 (32%)]	Loss: 37.326408
Train Epoch: 2 [3840/8000 (48%)]	Loss: 36.102455
Train Epoch: 2 [5120/8000 (64%)]	Loss: 35.597836
Train Epoch: 2 [6400/8000 (80%)]	Loss: 34.990261
Train Epoch: 2 [7680/8000 (96%)]	Loss: 34.360985
====> Epoch: 2 Average loss: 36.4155
Train Epoch: 3 [0/8000 (0%)]	Loss: 34.524200
Train Epoch: 3 [1280/8000 (16%)]	Loss: 34.207508
Train Epoch: 3 [2560/8000 (32%)]	Loss: 33.857361
Train Epoch: 3 [3840/8000 (48%)]	Loss: 33.793507
Train Epoch: 3 [5120/8000 (64%)]	Loss: 33.989220
Train E

Train Epoch: 22 [6400/8000 (80%)]	Loss: 31.002998
Train Epoch: 22 [7680/8000 (96%)]	Loss: 31.257687
====> Epoch: 22 Average loss: 31.2449
Train Epoch: 23 [0/8000 (0%)]	Loss: 31.386927
Train Epoch: 23 [1280/8000 (16%)]	Loss: 31.034521
Train Epoch: 23 [2560/8000 (32%)]	Loss: 31.336792
Train Epoch: 23 [3840/8000 (48%)]	Loss: 31.313383
Train Epoch: 23 [5120/8000 (64%)]	Loss: 31.017269
Train Epoch: 23 [6400/8000 (80%)]	Loss: 31.128880
Train Epoch: 23 [7680/8000 (96%)]	Loss: 31.526924
====> Epoch: 23 Average loss: 31.1897
Train Epoch: 24 [0/8000 (0%)]	Loss: 31.333666
Train Epoch: 24 [1280/8000 (16%)]	Loss: 31.354042
Train Epoch: 24 [2560/8000 (32%)]	Loss: 31.074629
Train Epoch: 24 [3840/8000 (48%)]	Loss: 31.058277
Train Epoch: 24 [5120/8000 (64%)]	Loss: 31.226425
Train Epoch: 24 [6400/8000 (80%)]	Loss: 30.920198
Train Epoch: 24 [7680/8000 (96%)]	Loss: 30.927282
====> Epoch: 24 Average loss: 31.1282
Train Epoch: 25 [0/8000 (0%)]	Loss: 31.069897
Train Epoch: 25 [1280/8000 (16%)]	Loss: 31.13264

Train Epoch: 44 [1280/8000 (16%)]	Loss: 30.636246
Train Epoch: 44 [2560/8000 (32%)]	Loss: 30.380066
Train Epoch: 44 [3840/8000 (48%)]	Loss: 30.260462
Train Epoch: 44 [5120/8000 (64%)]	Loss: 30.643415
Train Epoch: 44 [6400/8000 (80%)]	Loss: 30.413063
Train Epoch: 44 [7680/8000 (96%)]	Loss: 30.506575
====> Epoch: 44 Average loss: 30.4689
Train Epoch: 45 [0/8000 (0%)]	Loss: 30.496719
Train Epoch: 45 [1280/8000 (16%)]	Loss: 30.805012
Train Epoch: 45 [2560/8000 (32%)]	Loss: 30.479860
Train Epoch: 45 [3840/8000 (48%)]	Loss: 29.870306
Train Epoch: 45 [5120/8000 (64%)]	Loss: 30.277435
Train Epoch: 45 [6400/8000 (80%)]	Loss: 30.269686
Train Epoch: 45 [7680/8000 (96%)]	Loss: 30.485321
====> Epoch: 45 Average loss: 30.4502
Train Epoch: 46 [0/8000 (0%)]	Loss: 30.353239
Train Epoch: 46 [1280/8000 (16%)]	Loss: 30.411106
Train Epoch: 46 [2560/8000 (32%)]	Loss: 30.417006
Train Epoch: 46 [3840/8000 (48%)]	Loss: 30.377390
Train Epoch: 46 [5120/8000 (64%)]	Loss: 30.554312
Train Epoch: 46 [6400/8000 (80%)

Train Epoch: 65 [5120/8000 (64%)]	Loss: 30.261757
Train Epoch: 65 [6400/8000 (80%)]	Loss: 30.214964
Train Epoch: 65 [7680/8000 (96%)]	Loss: 30.428467
====> Epoch: 65 Average loss: 30.2000
Train Epoch: 66 [0/8000 (0%)]	Loss: 30.205038
Train Epoch: 66 [1280/8000 (16%)]	Loss: 30.331818
Train Epoch: 66 [2560/8000 (32%)]	Loss: 30.319281
Train Epoch: 66 [3840/8000 (48%)]	Loss: 30.250435
Train Epoch: 66 [5120/8000 (64%)]	Loss: 30.162100
Train Epoch: 66 [6400/8000 (80%)]	Loss: 30.575787
Train Epoch: 66 [7680/8000 (96%)]	Loss: 30.229120
====> Epoch: 66 Average loss: 30.1954
Train Epoch: 67 [0/8000 (0%)]	Loss: 30.468554
Train Epoch: 67 [1280/8000 (16%)]	Loss: 30.096970
Train Epoch: 67 [2560/8000 (32%)]	Loss: 29.976551
Train Epoch: 67 [3840/8000 (48%)]	Loss: 29.963739
Train Epoch: 67 [5120/8000 (64%)]	Loss: 29.879070
Train Epoch: 67 [6400/8000 (80%)]	Loss: 30.164143
Train Epoch: 67 [7680/8000 (96%)]	Loss: 30.291126
====> Epoch: 67 Average loss: 30.1869
Train Epoch: 68 [0/8000 (0%)]	Loss: 30.12374

Train Epoch: 87 [1280/8000 (16%)]	Loss: 30.097862
Train Epoch: 87 [2560/8000 (32%)]	Loss: 29.968924
Train Epoch: 87 [3840/8000 (48%)]	Loss: 30.154158
Train Epoch: 87 [5120/8000 (64%)]	Loss: 30.106409
Train Epoch: 87 [6400/8000 (80%)]	Loss: 30.150116
Train Epoch: 87 [7680/8000 (96%)]	Loss: 29.816467
====> Epoch: 87 Average loss: 30.0200
Train Epoch: 88 [0/8000 (0%)]	Loss: 29.852245
Train Epoch: 88 [1280/8000 (16%)]	Loss: 29.997995
Train Epoch: 88 [2560/8000 (32%)]	Loss: 30.010605
Train Epoch: 88 [3840/8000 (48%)]	Loss: 29.869608
Train Epoch: 88 [5120/8000 (64%)]	Loss: 29.999699
Train Epoch: 88 [6400/8000 (80%)]	Loss: 29.869789
Train Epoch: 88 [7680/8000 (96%)]	Loss: 29.824991
====> Epoch: 88 Average loss: 30.0172
Train Epoch: 89 [0/8000 (0%)]	Loss: 29.926689
Train Epoch: 89 [1280/8000 (16%)]	Loss: 30.235100
Train Epoch: 89 [2560/8000 (32%)]	Loss: 30.369614
Train Epoch: 89 [3840/8000 (48%)]	Loss: 29.906677
Train Epoch: 89 [5120/8000 (64%)]	Loss: 29.949619
Train Epoch: 89 [6400/8000 (80%)

...

Train Epoch: 171 [3840/8000 (48%)]	Loss: 29.691778
Train Epoch: 171 [5120/8000 (64%)]	Loss: 29.820354
Train Epoch: 171 [6400/8000 (80%)]	Loss: 29.697687
Train Epoch: 171 [7680/8000 (96%)]	Loss: 29.529900
====> Epoch: 171 Average loss: 29.8322
Train Epoch: 172 [0/8000 (0%)]	Loss: 30.050375
Train Epoch: 172 [1280/8000 (16%)]	Loss: 29.819826
Train Epoch: 172 [2560/8000 (32%)]	Loss: 29.893105
Train Epoch: 172 [3840/8000 (48%)]	Loss: 29.835381
Train Epoch: 172 [5120/8000 (64%)]	Loss: 29.854538
Train Epoch: 172 [6400/8000 (80%)]	Loss: 29.700729
Train Epoch: 172 [7680/8000 (96%)]	Loss: 29.893032
====> Epoch: 172 Average loss: 29.8292
Train Epoch: 173 [0/8000 (0%)]	Loss: 29.822430
Train Epoch: 173 [1280/8000 (16%)]	Loss: 30.048073
Train Epoch: 173 [2560/8000 (32%)]	Loss: 29.792807
Train Epoch: 173 [3840/8000 (48%)]	Loss: 29.933641
Train Epoch: 173 [5120/8000 (64%)]	Loss: 30.016602
Train Epoch: 173 [6400/8000 (80%)]	Loss: 29.747728
Train Epoch: 173 [7680/8000 (96%)]	Loss: 29.592339
====> Epoch:

Train Epoch: 192 [3840/8000 (48%)]	Loss: 30.103653
Train Epoch: 192 [5120/8000 (64%)]	Loss: 29.973534
Train Epoch: 192 [6400/8000 (80%)]	Loss: 30.001276
Train Epoch: 192 [7680/8000 (96%)]	Loss: 29.812340
====> Epoch: 192 Average loss: 29.8118
Train Epoch: 193 [0/8000 (0%)]	Loss: 29.887241
Train Epoch: 193 [1280/8000 (16%)]	Loss: 29.800154
Train Epoch: 193 [2560/8000 (32%)]	Loss: 29.862894
Train Epoch: 193 [3840/8000 (48%)]	Loss: 30.167154
Train Epoch: 193 [5120/8000 (64%)]	Loss: 29.892557
Train Epoch: 193 [6400/8000 (80%)]	Loss: 29.848461
Train Epoch: 193 [7680/8000 (96%)]	Loss: 29.772787
====> Epoch: 193 Average loss: 29.8129
Train Epoch: 194 [0/8000 (0%)]	Loss: 30.023052
Train Epoch: 194 [1280/8000 (16%)]	Loss: 29.723381
Train Epoch: 194 [2560/8000 (32%)]	Loss: 29.811617
Train Epoch: 194 [3840/8000 (48%)]	Loss: 29.659929
Train Epoch: 194 [5120/8000 (64%)]	Loss: 29.900434
Train Epoch: 194 [6400/8000 (80%)]	Loss: 29.959351
Train Epoch: 194 [7680/8000 (96%)]	Loss: 30.050060
====> Epoch:

Train Epoch: 213 [3840/8000 (48%)]	Loss: 30.016869
Train Epoch: 213 [5120/8000 (64%)]	Loss: 29.846630
Train Epoch: 213 [6400/8000 (80%)]	Loss: 29.708797
Train Epoch: 213 [7680/8000 (96%)]	Loss: 29.815838
====> Epoch: 213 Average loss: 29.8011
Train Epoch: 214 [0/8000 (0%)]	Loss: 29.759727
Train Epoch: 214 [1280/8000 (16%)]	Loss: 29.817102
Train Epoch: 214 [2560/8000 (32%)]	Loss: 29.760227
Train Epoch: 214 [3840/8000 (48%)]	Loss: 29.571650
Train Epoch: 214 [5120/8000 (64%)]	Loss: 30.152962
Train Epoch: 214 [6400/8000 (80%)]	Loss: 29.853821
Train Epoch: 214 [7680/8000 (96%)]	Loss: 29.901327
====> Epoch: 214 Average loss: 29.8008
Train Epoch: 215 [0/8000 (0%)]	Loss: 29.709595
Train Epoch: 215 [1280/8000 (16%)]	Loss: 30.049688
Train Epoch: 215 [2560/8000 (32%)]	Loss: 29.787767
Train Epoch: 215 [3840/8000 (48%)]	Loss: 29.884525
Train Epoch: 215 [5120/8000 (64%)]	Loss: 29.654650
Train Epoch: 215 [6400/8000 (80%)]	Loss: 30.027760
Train Epoch: 215 [7680/8000 (96%)]	Loss: 29.874723
====> Epoch:

Train Epoch: 234 [3840/8000 (48%)]	Loss: 29.908997
Train Epoch: 234 [5120/8000 (64%)]	Loss: 29.866787
Train Epoch: 234 [6400/8000 (80%)]	Loss: 29.827703
Train Epoch: 234 [7680/8000 (96%)]	Loss: 29.581121
====> Epoch: 234 Average loss: 29.7829
Train Epoch: 235 [0/8000 (0%)]	Loss: 29.701721
Train Epoch: 235 [1280/8000 (16%)]	Loss: 30.068953
Train Epoch: 235 [2560/8000 (32%)]	Loss: 29.704588
Train Epoch: 235 [3840/8000 (48%)]	Loss: 29.709436
Train Epoch: 235 [5120/8000 (64%)]	Loss: 29.659035
Train Epoch: 235 [6400/8000 (80%)]	Loss: 29.860847
Train Epoch: 235 [7680/8000 (96%)]	Loss: 29.674393
====> Epoch: 235 Average loss: 29.7795
Train Epoch: 236 [0/8000 (0%)]	Loss: 29.649414
Train Epoch: 236 [1280/8000 (16%)]	Loss: 29.797354
Train Epoch: 236 [2560/8000 (32%)]	Loss: 29.569967
Train Epoch: 236 [3840/8000 (48%)]	Loss: 29.799479
Train Epoch: 236 [5120/8000 (64%)]	Loss: 29.751755
Train Epoch: 236 [6400/8000 (80%)]	Loss: 29.808723
Train Epoch: 236 [7680/8000 (96%)]	Loss: 29.927212
====> Epoch:

KeyboardInterrupt: 

In [ ]:
print(gradients_before_burnin[:D].sum())
print(gradients_before_burnin[D:].sum())
sns.heatmap(gradients_before_burnin.clone().detach().cpu().numpy()[:, np.newaxis])


In [ ]:
print(gradient_post_burn_in[:D].sum())
print(gradient_post_burn_in[D:].sum())
sns.heatmap(gradient_post_burn_in.clone().detach().cpu().numpy()[:, np.newaxis])

In [ ]:
print(subset_indices_before_burnin[:D].sum())
print(subset_indices_before_burnin[D:].sum())
sns.heatmap(subset_indices_before_burnin.clone().detach().cpu().numpy()[:, np.newaxis])

In [ ]:
print(subset_indices_post_burnin[:D].sum())
print(subset_indices_post_burnin[(D):].sum())
sns.heatmap(subset_indices_post_burnin.clone().detach().cpu().numpy()[:, np.newaxis])

In [ ]:
subset_indices_post_burnin[:(D)].sum() - subset_indices_post_burnin[(D):].sum()

The new model


In [ ]:
vae_gumbel_truncated = VAE_Gumbel_NInstaState(2*D, 100, 20, k = 3*z_size, t = global_t)
vae_gumbel_truncated.to(device)
vae_gumbel_trunc_optimizer = torch.optim.Adam(vae_gumbel_truncated.parameters(), 
                                                lr=lr, 
                                                betas = (b1,b2))



In [ ]:
gradients_before_burnin = torch.zeros(train_data.shape[1]).to(device)
gradient_post_burn_in = torch.zeros(train_data.shape[1]).to(device)
subset_indices_before_burnin = torch.zeros(train_data.shape[1]).to(device)
subset_indices_post_burnin = torch.zeros(train_data.shape[1]).to(device)

for epoch in range(1, n_epochs+1):
    grads=train_truncated_with_gradients_gumbel_state(train_data, vae_gumbel_truncated, 
                                                      vae_gumbel_trunc_optimizer, 
                                                      epoch, 
                                                      batch_size, 
                                                      Dim = 2*D)
    
    vae_gumbel_truncated.t = max(0.01, vae_gumbel_truncated.t * 0.99)
    if epoch <=(n_epochs//5*4):
        gradients_before_burnin += grads
        with torch.no_grad():
            subset_indices_before_burnin += sample_subset(vae_gumbel_truncated.logit_enc, 
                                                          vae_gumbel_truncated.k, 
                                                          vae_gumbel_truncated.t).view(-1)
    if epoch == (n_epochs//5*4):
        print("BURN IN DEBUG")
        vae_gumbel_truncated.set_burned_in()
        #vae_gumbel_truncated.t /= 10
        print("Going post burn in")
    if epoch > (n_epochs//5*4):
        gradient_post_burn_in += grads
        with torch.no_grad():
            subset_indices_post_burnin += sample_subset(vae_gumbel_truncated.logit_enc, 
                                                        vae_gumbel_truncated.k, 
                                                        vae_gumbel_truncated.t).view(-1)

In [ ]:
print(gradients_before_burnin[:(D)].mean())
print(gradients_before_burnin[(D):].mean())
sns.heatmap(gradients_before_burnin.clone().detach().cpu().numpy()[:, np.newaxis])


In [ ]:
print(gradient_post_burn_in[:D].mean())
print(gradient_post_burn_in[D:].mean())
sns.heatmap(gradient_post_burn_in.clone().detach().cpu().numpy()[:, np.newaxis])

In [ ]:
print(subset_indices_before_burnin[:D].sum())
print(subset_indices_before_burnin[D:].sum())
sns.heatmap(subset_indices_before_burnin.clone().detach().cpu().numpy()[:, np.newaxis])

In [ ]:
print(subset_indices_post_burnin[:D].sum())
print(subset_indices_post_burnin[D:].sum())
sns.heatmap(subset_indices_post_burnin.clone().detach().cpu().numpy()[:, np.newaxis])

In [ ]:
(subset_indices_post_burnin[:(D)] > 0).sum()

In [ ]:
(subset_indices_post_burnin[(D):] > 0).sum()

In [ ]:
subset_indices_post_burnin[:(D)]

In [ ]:
subset_indices_post_burnin[(D):]

In [ ]:
subset_indices_post_burnin[:(D)].sum() - subset_indices_post_burnin[(D):].sum()

In [ ]:
top_ind = torch.argsort(sample_subset(vae_gumbel_truncated.logit_enc, 
                                                        vae_gumbel_truncated.k, 
                                                        vae_gumbel_truncated.t).view(-1), 
                        descending = True)[:vae_gumbel_truncated.k]

In [ ]:
sum(top_ind < 30)

In [ ]:
sum(top_ind >= 30)